## Setup

Instalando o [TensorFlow Datasets](https://tensorflow.org/datasets) para carregar o conjunto de dados e o [TensorFlow Text](https://www.tensorflow.org/text) para o pré-processamento de texto:



In [ ]:
# Install the most re version of TensorFlow to use the improved
# masking support for `tf.keras.layers.MultiHeadAttention`.
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install protobuf~=3.20.3
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow

Importe os módulos necessários:

In [ ]:
import logging
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

import tensorflow_text

## Manipulação de Dados

Esta seção faz o download do conjunto de dados e do tokenizador de subpalavras, a partir [deste tutorial](https://www.tensorflow.org/text/guide/subwords_tokenizer), e depois organiza tudo em um `tf.data.Dataset` para o treinamento.

<section class="expandable tfo-display-only-on-site">
<button type="button" class="button-red button expand-control">Alternar seção</button>

### Download do dataset

Usamos o TensorFlow Datasets para carregar o [conjunto de dados de tradução Português-Inglês](https://www.tensorflow.org/datasets/catalog/ted_hrlr_translate#ted_hrlr_translatept_to_en) do TED Talks Open Translation Project. Esse conjunto de dados contém aproximadamente 52.000 exemplos de treino, 1.200 para validação e 1.800 exemplos de teste.

In [ ]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en',
                               with_info=True,
                               as_supervised=True)

train_examples, val_examples = examples['train'], examples['validation']

O objeto `tf.data.Dataset` retornado pelo TensorFlow Datasets gera pares de exemplos de texto:

In [ ]:
for pt_examples, en_examples in train_examples.batch(3).take(1):
  print('> Exemplos em Português:')
  for pt in pt_examples.numpy():
    print(pt.decode('utf-8'))
  print()

  print('> Exemplos em Inglês:')
  for en in en_examples.numpy():
    print(en.decode('utf-8'))

### Configurar o tokenizador

Agora que carregamos o conjunto de dados, precisamos tokenizar o texto, de forma que cada elemento seja representado como um [token](https://developers.google.com/machine-learning/glossary#token) ou ID de token (uma representação numérica).

A tokenização é o processo de dividir o texto em "tokens". Dependendo do tokenizador, esses tokens podem representar pedaços de sentenças, palavras, subpalavras ou caracteres. Para saber mais sobre tokenização, visitamos [este guia](https://www.tensorflow.org/text/guide/tokenizers).

In [ ]:
model_name = 'ted_hrlr_translate_pt_en_converter'
tf.keras.utils.get_file(
    f'{model_name}.zip',
    f'https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip',
    cache_dir='.', cache_subdir='', extract=True
)

In [ ]:
tokenizers = tf.saved_model.load(model_name)

In [ ]:
[item for item in dir(tokenizers.en) if not item.startswith('_')]

In [ ]:
print('> This is a batch of strings:')
for en in en_examples.numpy():
  print(en.decode('utf-8'))

In [ ]:
encoded = tokenizers.en.tokenize(en_examples)

print('> This is a padded-batch of token IDs:')
for row in encoded.to_list():
  print(row)

In [ ]:
round_trip = tokenizers.en.detokenize(encoded)

print('> This is human-readable text:')
for line in round_trip.numpy():
  print(line.decode('utf-8'))

In [ ]:
print('> This is the text split into tokens:')
tokens = tokenizers.en.lookup(encoded)
tokens

A saída demonstra o aspecto de "subpalavra" da tokenização de subpalavras.

Por exemplo, a palavra `'searchability'` é decomposta em `'search'` e `'##ability'`, enquanto a palavra `'serendipity'` é decomposta em `'s'`, `'##ere'`, `'##nd'`, `'##ip'` e `'##ity'`.

Observe que o texto tokenizado inclui os tokens `'[START]'` e `'[END]'`.

In [ ]:
lengths = []

for pt_examples, en_examples in train_examples.batch(1024):
  pt_tokens = tokenizers.pt.tokenize(pt_examples)
  lengths.append(pt_tokens.row_lengths())

  en_tokens = tokenizers.en.tokenize(en_examples)
  lengths.append(en_tokens.row_lengths())
  print('.', end='', flush=True)

In [ ]:
all_lengths = np.concatenate(lengths)

plt.hist(all_lengths, np.linspace(0, 500, 101))
plt.ylim(plt.ylim())
max_length = max(all_lengths)
plt.plot([max_length, max_length], plt.ylim())
plt.title(f'Maximum tokens per example: {max_length}');

### Configurar um pipeline de dados com `tf.data`

A seguinte função recebe lotes de texto como entrada e os converte para um formato adequado para treinamento.

1. Ela tokeniza os textos em lotes irregulares.
2. Ela corta cada um para não ter mais do que `MAX_TOKENS`.
3. Ela separa os tokens de destino (inglês) em entradas e rótulos, deslocando-os uma etapa para que, em cada posição de entrada, o `rótulo` seja o ID do próximo token.
4. Ela converte os `RaggedTensor`s em `Tensor`s densos com preenchimento.
5. Ela retorna um par `(entradas, rótulos)`.

In [ ]:
MAX_TOKENS=64
def prepare_batch(pt, en):
    pt = tokenizers.pt.tokenize(pt)      # Output is ragged.
    pt = pt[:, :MAX_TOKENS]    # Trim to MAX_TOKENS.
    pt = pt.to_tensor()  # Convert to 0-padded dense Tensor

    en = tokenizers.en.tokenize(en)
    en = en[:, :(MAX_TOKENS+1)]
    en_inputs = en[:, :-1].to_tensor()  # Drop the [END] tokens
    en_labels = en[:, 1:].to_tensor()   # Drop the [START] tokens

    return (pt, en_inputs), en_labels

A função abaixo converte um conjunto de dados de exemplos de texto em dados em lotes para treinamento.

1. Ela tokeniza o texto e filtra as sequências que são muito longas.  
   (O uso de `batch`/`unbatch` é incluído porque o tokenizador é muito mais eficiente em grandes lotes).
2. O método `cache` garante que esse trabalho seja executado apenas uma vez.
3. Em seguida, `shuffle` e `dense_to_ragged_batch` randomizam a ordem e montam lotes de exemplos.
4. Por fim, `prefetch` executa o conjunto de dados em paralelo com o modelo para garantir que os dados estejam disponíveis quando necessário. Veja [Melhores desempenhos com o `tf.data`](https://www.tensorflow.org/guide/data_performance.ipynb) para mais detalhes.

In [ ]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [ ]:
def make_batches(ds):
  return (
      ds
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(prepare_batch, tf.data.AUTOTUNE)
      .prefetch(buffer_size=tf.data.AUTOTUNE))

 </section>

## Dataset de Teste

In [ ]:
# Create training and validation set batches.
train_batches = make_batches(train_examples)
val_batches = make_batches(val_examples)

In [ ]:
for (pt, en), en_labels in train_batches.take(1):
  break

print(pt.shape)
print(en.shape)
print(en_labels.shape)

The `en` and `en_labels` are the same, just shifted by 1:

In [ ]:
print(en[0][:10])
print(en_labels[0][:10])

## Definir os componentes

In [ ]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1)

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
#@title
pos_encoding = positional_encoding(length=2048, depth=512)

# Check the shape.
print(pos_encoding.shape)

# Plot the dimensions.
plt.pcolormesh(pos_encoding.numpy().T, cmap='RdBu')
plt.ylabel('Depth')
plt.xlabel('Position')
plt.colorbar()
plt.show()

Por definição, esses vetores se alinham bem com vetores próximos ao longo do eixo de posição. Abaixo, os vetores de codificação de posição são normalizados, e o vetor da posição `1000` é comparado, por meio do produto escalar, a todos os outros:

In [ ]:
#@title
pos_encoding/=tf.norm(pos_encoding, axis=1, keepdims=True)
p = pos_encoding[1000]
dots = tf.einsum('pd,d -> p', pos_encoding, p)
plt.subplot(2,1,1)
plt.plot(dots)
plt.ylim([0,1])
plt.plot([950, 950, float('nan'), 1050, 1050],
         [0,1,float('nan'),0,1], color='k', label='Zoom')
plt.legend()
plt.subplot(2,1,2)
plt.plot(dots)
plt.xlim([950, 1050])
plt.ylim([0,1])


In [ ]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x


In [ ]:
embed_pt = PositionalEmbedding(vocab_size=tokenizers.pt.get_vocab_size().numpy(), d_model=248)
embed_en = PositionalEmbedding(vocab_size=tokenizers.en.get_vocab_size().numpy(), d_model=248)

pt_emb = embed_pt(pt)
en_emb = embed_en(en)

In [ ]:
en_emb._keras_mask